In [1]:
__shop__ = 'AINLP'
__link__ = 'http://item.taobao.com/item.htm?id=637341876946'
__date__ = 2022 / 4 / 23

In [2]:
try:
    import torch
except ImportError:
    os.system('pip install torch -i https://pypi.tuna.tsinghua.edu.cn/simple')
    import torch
try:
    from transformers import BertTokenizer, BertModel
except ImportError:
    os.system(
        'pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple')
    from transformers import BertTokenizer, BertModel
import os

In [13]:
model_path = 'pretrain/output/checkpoint-3000'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

Some weights of the model checkpoint at pretrain/output/checkpoint-3000 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at pretrain/output/checkpoint-3000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.p

In [14]:
word_embeddings = model.embeddings.word_embeddings.weight

In [15]:
word_embeddings

Parameter containing:
tensor([[ 0.0258, -0.0047, -0.0044,  ...,  0.0296,  0.0238,  0.0196],
        [ 0.0103, -0.0124,  0.0065,  ...,  0.0124,  0.0265,  0.0336],
        [ 0.0214, -0.0018,  0.0185,  ...,  0.0074,  0.0271,  0.0054],
        ...,
        [ 0.0076,  0.0190,  0.0258,  ...,  0.0238,  0.0653,  0.0428],
        [ 0.0175,  0.0163,  0.0251,  ...,  0.0329,  0.0061,  0.0211],
        [ 0.0214,  0.0199, -0.0136,  ..., -0.0060,  0.0056, -0.0059]],
       requires_grad=True)

In [16]:
def get_embeeding(sentence):
    embeddings = []
    tokenized = tokenizer.batch_encode_plus([sentence])['input_ids']
    embedded = word_embeddings[tokenized].mean(axis=0).unsqueeze(0)
    return embedded

In [17]:
@torch.no_grad()
def compute_sim(sentence_a, sentence_b):
    embed_a = get_embeeding(sentence_a)
    embed_b = get_embeeding(sentence_b)
    embedding_a_tensor_norm = embed_a / embed_a.norm(dim=1)[:, None]
    embedding_b_tensor_norm = embed_b / embed_b.norm(dim=1)[:, None]
    sim = torch.mm(embedding_a_tensor_norm, embedding_b_tensor_norm.transpose(0, 1)).squeeze(0)
    return sim.numpy()[0]

In [18]:
compute_sim('Perovskite', 'photovoltaic')

0.49921876

In [19]:
compute_sim('Perovskite', '光伏')

0.51279104